In [1]:
!pip install langchain langchain-groq langchain-exa langchain-community langgraph pyowm -U --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 17.0 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
EXA_API_KEY = userdata.get("EXA_API_KEY")
GROQ_API_KEY = userdata.get("GROQ_API_KEY")
OPENWEATHERMAP_API_KEY = userdata.get("OPENWEATHERMAP_API_KEY")

In [3]:
from langchain_groq import ChatGroq
from langchain_exa import ExaSearchResults, ExaFindSimilarResults
from langgraph.prebuilt import create_react_agent
from langchain_community.utilities import OpenWeatherMapAPIWrapper

In [12]:
# Initialize the language model
llm = ChatGroq(
  model="llama3-70b-8192",
  temperature=0.2,
  api_key=GROQ_API_KEY
)

In [6]:
# Initialize Exa Tools - for web search
exa_search = ExaSearchResults(
    exa_api_key=EXA_API_KEY,
    max_results=5,
)

exa_find_similar=ExaFindSimilarResults(
    exa_api_key=EXA_API_KEY,
    max_results=5,
)

# Initialize OpenWeatherMap Tool
weather = OpenWeatherMapAPIWrapper(
    openweathermap_api_key=OPENWEATHERMAP_API_KEY
)

In [7]:
tools = [exa_search, exa_find_similar, weather.run]

In [13]:
agent = create_react_agent(
    llm,
    tools=tools
)

In [10]:
city_name = "New York"

In [14]:
input_message = {
    "role": "user",
    "content": f"""
I'm travelling to {city_name} today. I want you to check today's weather and suggest either indoor or outdoor dining.
Pick 3 iconic local dishes.
Find top-rated restaurants serving these dishes.
And finally create a delightful one-day "foodie tour", crafting breakfast, lunch, and dinner
narratives that factor in weather conditions.
""",
}

for step in agent.stream(
    {"messages": [input_message]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================


I'm travelling to New York today. I want you to check today's weather and suggest either indoor or outdoor dining.
Pick 3 iconic local dishes.
Find top-rated restaurants serving these dishes.
And finally create a delightful one-day "foodie tour", crafting breakfast, lunch, and dinner
narratives that factor in weather conditions.

================================== Ai Message ==================================
Tool Calls:
  run (4kvrqkp3y)
 Call ID: 4kvrqkp3y
  Args:
    location: New York
================================= Tool Message =================================
Name: run

In New York, the current weather is as follows:
Detailed status: broken clouds
Wind speed: 6.17 m/s, direction: 210°
Humidity: 49%
Temperature: 
  - Current: 31.0°C
  - High: 32.38°C
  - Low: 29.42°C
  - Feels like: 32.42°C
Rain: {}
Heat index: None
Cloud cover: 75%
================================== Ai Message ==================